In [1]:
# ==============================================================================
# LCP_OneToAll_Parallel_FINAL.py
# CORRECCIÓN DEFINITIVA: Se añade una función de "calentamiento" (warm-up)
# para las funciones de Numba. Esto pre-compila el código JIT en el proceso
# principal para evitar conflictos y caídas del kernel durante la paralelización.
# ==============================================================================

# --- 1. IMPORTACIONES ---
import os
import numpy as np
import rasterio
import math
import logging
import time
import threading
import psutil
from datetime import datetime
from tqdm import tqdm
import fiona
from fiona.crs import CRS
from shapely.geometry import Point, LineString, mapping, shape
from shapely.ops import unary_union
from numba import njit
from skimage.draw import disk
from rasterio.enums import Resampling
from rasterio.features import rasterize
from joblib import Parallel, delayed


In [2]:

# ==============================================================================
# --- 2. CONFIGURACIÓN DEL PROYECTO Y PARÁMETROS ---
# ==============================================================================
# --- 1. PARÁMETROS CONFIGURABLES POR EL USUARIO ---
BASE_DIR = os.getcwd()

# --- 1.1 Preguntar a usuario nombre de carpeta con archivos
"""
Determinar carpeta con pop-up nativo del SO
Problema -> La ventana de eleccion de carpeta aparece por debajo de la ventana actual
data_path = askdirectory(title='Seleccionar carpeta con archivos a usar', initialdir = BASE_DIR) 
DATA_DIR = os.path.abspath(data_path)
"""
# Pregunta por nombre de carpeta en la línea misma
carpeta_user = input("Cuál es el nombre de la carpeta con tus datos? (Tiene que estar dentro de /output): ")
DATA_DIR = os.path.join(BASE_DIR, 'data', carpeta_user)
print(f"Recuerda que tus archivos deben llamarse 'cost.tif', 'points.shp' y 'area-mask.shp'\nLa carpeta a usar será {carpeta_user}")
COST_RASTER_PATH = os.path.join(DATA_DIR, 'cost.tif')
ALL_POINTS_SHAPEFILE = os.path.join(DATA_DIR, 'points.shp')
MASK_SHAPEFILE_PATH = os.path.join(DATA_DIR, 'area-mask.shp')
ORIGIN_POINT_ID = input("Desde que punto debería realizarse el análisis?: ")
ID_FIELD_NAME = input("Cuál es el nombre de la columna de id en tu archivo?")
opcionRutas = input("Juntar y exportar las rutas a un solo archivo posterior al análisis? (Y/N)")
DOWNSAMPLING_FACTORS = [32, 20, 10]
CORRIDOR_BUFFER_PIXELS = 150
HEURISTIC_WEIGHT = 1.0
N_JOBS = -2


Recuerda que tus archivos deben llamarse 'cost.tif', 'points.shp' y 'area-mask.shp'
La carpeta a usar será 


In [ ]:

# ==============================================================================
# --- 3. FUNCIONES (Sin cambios en su lógica interna) ---
# ==============================================================================
class TqdmLoggingHandler(logging.Handler):
    def __init__(self, level=logging.NOTSET): super().__init__(level)
    def emit(self, record):
        try: msg = self.format(record); tqdm.write(msg); self.flush()
        except Exception: self.handleError(record)

def load_all_points(shapefile_path, id_field):
    points = {}; crs = None
    try:
        with fiona.open(shapefile_path, 'r') as c:
            crs = c.crs;
            if len(c) == 0: logging.error(f"'{os.path.basename(shapefile_path)}' está vacío."); return None, None
            for f in c: points[int(f['properties'][id_field])] = f['geometry']['coordinates']
        return points, crs
    except Exception as e: logging.error(f"Error cargando puntos: {e}"); return None, None

def load_polygon_geometry(shapefile_path):
    if not shapefile_path or not os.path.exists(shapefile_path): return None
    try:
        with fiona.open(shapefile_path, 'r') as c:
            if len(c) == 0: logging.warning(f"'{os.path.basename(shapefile_path)}' de máscara está vacío."); return None
            return unary_union([shape(f['geometry']) for f in c])
    except Exception as e: logging.error(f"Error cargando polígono: {e}"); return None

def world_to_pixel(transform, x, y): col, row = ~transform * (x, y); return int(row), int(col)
def create_mask_from_vector(vector_path, raster_src):
    if not vector_path or not os.path.exists(vector_path):
        logging.warning("No se encontró archivo de máscara."); return None
    with fiona.open(vector_path, "r") as vf:
        if vf.crs != raster_src.crs:
            logging.critical(f"CRS de máscara ({vf.crs}) y ráster ({raster_src.crs}) no coinciden."); return None
        shapes = [f["geometry"] for f in vf]
    return rasterize(shapes, out_shape=raster_src.shape, transform=raster_src.transform, fill=0, all_touched=True, dtype=np.uint8).astype(bool)

@njit
def heuristic_numba(r1, c1, r2, c2, dx, dy): return math.sqrt((r2 - r1)**2 + (c2 - c1)**2) * math.sqrt(dx*dy)
@njit
def a_star_numba_compiled(cost_array, nodata_value, start_pixel, end_pixel, dx, dy, weight, search_mask):
    height, width = cost_array.shape; g_cost = np.full(cost_array.shape, np.inf); came_from = np.full(cost_array.shape, -1, dtype=np.int16)
    open_set = np.zeros((1, 4)); h_initial = heuristic_numba(start_pixel[0], start_pixel[1], end_pixel[0], end_pixel[1], dx, dy)
    open_set[0] = [h_initial*weight, 0.0, start_pixel[0], start_pixel[1]]; g_cost[start_pixel] = 0; path_found = False
    while open_set.shape[0] > 0:
        min_idx = np.argmin(open_set[:, 0]); f, g, r, c = open_set[min_idx]; current_pos = (int(r), int(c))
        if open_set.shape[0] == 1: open_set = np.zeros((0, 4))
        else: open_set = np.vstack((open_set[:min_idx], open_set[min_idx + 1:]))
        if current_pos == end_pixel: path_found = True; break
        if g > g_cost[current_pos]: continue
        cost_current = cost_array[current_pos]
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                if dr == 0 and dc == 0: continue
                neighbor_pos = (current_pos[0] + dr, current_pos[1] + dc)
                if not (0 <= neighbor_pos[0] < height and 0 <= neighbor_pos[1] < width): continue
                if not search_mask[neighbor_pos]: continue
                cost_neighbor = cost_array[neighbor_pos];
                if cost_neighbor == nodata_value: continue
                dist_m = math.sqrt((dr*dy)**2 + (dc*dx)**2); avg_cost = (cost_current + cost_neighbor) / 2.0
                tentative_g_cost = g + (avg_cost*dist_m)
                if tentative_g_cost < g_cost[neighbor_pos]:
                    direction = (dr+1)*3 + (dc+1); came_from[neighbor_pos] = direction; g_cost[neighbor_pos] = tentative_g_cost
                    h = heuristic_numba(neighbor_pos[0], neighbor_pos[1], end_pixel[0], end_pixel[1], dx, dy)
                    new_f_cost = tentative_g_cost + (h*weight)
                    new_entry = np.array([[new_f_cost, tentative_g_cost, neighbor_pos[0], neighbor_pos[1]]])
                    open_set = np.vstack((open_set, new_entry))
    return path_found, came_from, g_cost

@njit
def reconstruct_path_pixels_numba(came_from_array, start_pixel, end_pixel):
    path = np.zeros((came_from_array.size, 2), dtype=np.int32); current_pos_r, current_pos_c = end_pixel; count = 0; limit = came_from_array.size
    while (current_pos_r, current_pos_c) != start_pixel and count < limit:
        path[count] = np.array([current_pos_r, current_pos_c]); direction = came_from_array[current_pos_r, current_pos_c]
        if direction == -1: return None
        dc = (direction % 3) - 1; dr = (direction // 3) - 1; current_pos_r -= dr; current_pos_c -= dc; count += 1
    path[count] = np.array([start_pixel[0], start_pixel[1]]); return path[:count+1][::-1]

def create_low_res_data(src_transform, src_res, src_shape, cost_data_high_res, factor):
    low_res_data = cost_data_high_res[::factor, ::factor]
    low_res_transform = src_transform * src_transform.scale(factor, factor)
    return low_res_data, low_res_transform, src_res[0]*factor, src_res[1]*factor

def create_search_corridor(path_low_res, high_res_shape, factor, buffer_pixels):
    corridor_mask = np.zeros(high_res_shape, dtype=bool);
    if path_low_res is None: return corridor_mask
    for r_low, c_low in path_low_res:
        r_high = int(r_low*factor + factor/2); c_high = int(c_low*factor + factor/2)
        rr, cc = disk((r_high, c_high), buffer_pixels, shape=high_res_shape); corridor_mask[rr, cc] = True
    return corridor_mask
def save_path_to_shapefile(pixel_path, transform, crs, output_path):
    if pixel_path is None or not pixel_path.any():
        print(f"ADVERTENCIA: No se guardará {os.path.basename(output_path)}, ruta vacía.", flush=True); return
    world_coords = [transform * (p[1] + 0.5, p[0] + 0.5) for p in pixel_path]
    schema = {'geometry': 'LineString', 'properties': {'id': 'str'}}
    with fiona.open(output_path, 'w', 'ESRI Shapefile', schema, crs=CRS.from_wkt(crs.to_wkt()) if crs else None) as c:
        c.write({'geometry': mapping(LineString(world_coords)), 'properties': {'id': os.path.basename(output_path)}})



In [ ]:

# ==============================================================================
# --- 4. NUMBA WARM-UP Y FUNCIONES DE MONITOREO ---
# ==============================================================================

def warm_up_numba():
    """
    Ejecuta las funciones compiladas por Numba con datos de juguete para
    forzar la compilación JIT en el proceso principal antes de la paralelización.
    """
    log.info("Calentando funciones de Numba para evitar conflictos en paralelo...")
    # Datos de juguete
    dummy_cost = np.ones((3, 3), dtype=np.float32)
    dummy_mask = np.ones((3, 3), dtype=bool)
    start = (0, 0); end = (2, 2)
    
    # Calentar a_star
    _, came_from, _ = a_star_numba_compiled(dummy_cost, -9999, start, end, 1.0, 1.0, 1.0, dummy_mask)
    
    # Calentar reconstruct_path
    came_from[1, 1] = 4  # Dirección de (0,0) a (1,1)
    _ = reconstruct_path_pixels_numba(came_from, start, (1, 1))
    
    # Calentar heuristic
    _ = heuristic_numba(0,0,1,1,1,1)
    log.info("Calentamiento de Numba completado.")

def calculate_route_worker(args):
    """
    Función de trabajo que calcula la ruta para un único destino.
    Ahora asume que las funciones de Numba ya están compiladas.
    """
    dest_id, dest_coords, shared_data = args
    origin_id = shared_data['origin_id']
    print(f"--- Calculando ruta desde {origin_id} hasta {dest_id} ---", flush=True)
    try:
        with rasterio.open(shared_data['cost_raster_path']) as src:
            cost_data_high_res = src.read(1)
            search_mask_hr_user = None
            if shared_data['mask_shapefile_path']:
                search_mask_hr_user = create_mask_from_vector(shared_data['mask_shapefile_path'], src)
        start_pixel_hr = shared_data['start_pixel_hr']
        end_pixel_hr = world_to_pixel(shared_data['transform'], dest_coords[0], dest_coords[1])
        path_found_lr, path_pixels_lr, successful_factor, trans_low = False, None, None, None
        print(f"[{origin_id}->{dest_id}] -> FASE 1: Búsqueda en baja resolución...", flush=True)
        for factor in shared_data['downsampling_factors']:
            cost_data_lr, trans_lr, dx_lr, dy_lr = create_low_res_data(shared_data['transform'], shared_data['res'], shared_data['shape'], cost_data_high_res, factor)
            search_mask_lr = search_mask_hr_user[::factor, ::factor] if search_mask_hr_user is not None else np.ones(cost_data_lr.shape, dtype=bool)
            start_lr = (start_pixel_hr[0]//factor, start_pixel_hr[1]//factor); end_lr = (end_pixel_hr[0]//factor, end_pixel_hr[1]//factor)
            found, came_from_lr, _ = a_star_numba_compiled(cost_data_lr, shared_data['nodata'], start_lr, end_lr, dx_lr, abs(dy_lr), shared_data['heuristic_weight'], search_mask_lr)
            if found:
                print(f"  [{origin_id}->{dest_id}] ÉXITO con factor {factor}.", flush=True)
                path_found_lr, successful_factor, trans_low = True, factor, trans_lr
                path_pixels_lr = reconstruct_path_pixels_numba(came_from_lr, start_lr, end_lr); break
            else:
                print(f"  [{origin_id}->{dest_id}] FALLÓ con factor {factor}.", flush=True)
        path_found_hr, came_from_hr = False, None
        if path_found_lr:
            corridor_mask = create_search_corridor(path_pixels_lr, shared_data['shape'], successful_factor, shared_data['corridor_buffer_pixels'])
            final_mask = np.logical_and(corridor_mask, search_mask_hr_user) if search_mask_hr_user is not None else corridor_mask
            path_found_hr, came_from_hr, _ = a_star_numba_compiled(cost_data_high_res, shared_data['nodata'], start_pixel_hr, end_pixel_hr, shared_data['res'][0], abs(shared_data['res'][1]), shared_data['heuristic_weight'], final_mask)
        if not path_found_hr:
            if path_found_lr: print(f"  [{origin_id}->{dest_id}] Búsqueda en corredor falló.", flush=True)
            print(f"  [{origin_id}->{dest_id}] PLAN B: Realizando LCP sobre toda la máscara...", flush=True)
            fallback_mask = search_mask_hr_user if search_mask_hr_user is not None else np.ones_like(cost_data_high_res, dtype=bool)
            path_found_hr, came_from_hr, _ = a_star_numba_compiled(cost_data_high_res, shared_data['nodata'], start_pixel_hr, end_pixel_hr, shared_data['res'][0], abs(shared_data['res'][1]), shared_data['heuristic_weight'], fallback_mask)
        if path_found_hr:
            path_pixels_hr = reconstruct_path_pixels_numba(came_from_hr, start_pixel_hr, end_pixel_hr)
            if path_found_lr:
                p1_path = os.path.join(shared_data['session_output_dir'], f"ruta_fase1_desde_{origin_id}_a_{dest_id}.shp")
                save_path_to_shapefile(path_pixels_lr, trans_low, shared_data['crs'], p1_path)
            final_output_path = os.path.join(shared_data['session_output_dir'], f"ruta_final_desde_{origin_id}_a_{dest_id}.shp")
            save_path_to_shapefile(path_pixels_hr, shared_data['transform'], shared_data['crs'], final_output_path)
            print(f"-> [{origin_id}->{dest_id}] ÉXITO FINAL. Ruta guardada.", flush=True)
            return ("Éxito", origin_id, dest_id)
        else:
            print(f"-> [{origin_id}->{dest_id}] ERROR CRÍTICO: No se encontró ruta final.", flush=True)
            return ("Fallo", origin_id, dest_id)
    except Exception as e:
        print(f"-> [{origin_id}->{dest_id}] ERROR INESPERADO: {e}", flush=True)
        return ("Error", origin_id, dest_id, str(e))

def resource_monitor(stop_event, records_list, interval=1):
    process = psutil.Process(os.getpid())
    while not stop_event.is_set():
        cpu = psutil.cpu_percent(interval=None)
        mem = sum(p.memory_info().rss for p in [process] + process.children(recursive=True)) / (1024 * 1024)
        records_list.append({'cpu': cpu, 'mem_mb': mem})
        time.sleep(interval)



In [ ]:

# ==============================================================================
# --- 5. EJECUCIÓN DEL ANÁLISIS PARALELO ---
# ==============================================================================
if __name__ == '__main__':
    os.makedirs(BASE_PROCESSING_FOLDER, exist_ok=True)
    log_file_path = os.path.join(BASE_PROCESSING_FOLDER, "registro_maestro_procesamiento.log")
    log = logging.getLogger(); log.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    if not log.handlers:
        file_handler = logging.FileHandler(log_file_path, mode='a'); file_handler.setFormatter(formatter); log.addHandler(file_handler)
        tqdm_handler = TqdmLoggingHandler(); tqdm_handler.setFormatter(formatter); log.addHandler(tqdm_handler)

    log.info("="*50); log.info("INICIANDO NUEVA EJECUCIÓN PARALELA (UNO A TODOS)"); log.info("="*50)
    
    # --- [INICIO DE CORRECCIÓN CRÍTICA] ---
    # Calentar Numba ANTES de cualquier otra operación.
    warm_up_numba()
    # --- [FIN DE CORRECCIÓN CRÍTICA] ---

    monitoring_records = []
    try:
        all_points, points_crs = load_all_points(ALL_POINTS_SHAPEFILE, ID_FIELD_NAME)
        if not all_points: raise ValueError("No se pudieron cargar los puntos de entrada.")
        num_expected_routes = len(all_points) - 1
        session_to_resume = None
        try: existing_sessions = sorted([d for d in os.listdir(BASE_PROCESSING_FOLDER) if os.path.isdir(os.path.join(BASE_PROCESSING_FOLDER, d)) and d[:8].isdigit()])
        except FileNotFoundError: existing_sessions = []
        if existing_sessions:
            latest_session_name = existing_sessions[-1]
            latest_session_path = os.path.join(BASE_PROCESSING_FOLDER, latest_session_name)
            completed_routes = [f for f in os.listdir(latest_session_path) if f.startswith(f"ruta_final_desde_{ORIGIN_POINT_ID}_a_") and f.endswith(".shp")]
            if len(completed_routes) < num_expected_routes: session_to_resume = latest_session_name
        if session_to_resume:
            SESSION_OUTPUT_DIR = os.path.join(BASE_PROCESSING_FOLDER, session_to_resume)
            log.info(f"MODO REANUDACIÓN: Reanudando sesión incompleta '{session_to_resume}'.")
        else:
            if existing_sessions: log.info(f"La última sesión '{existing_sessions[-1]}' está completa.")
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            session_folder_name = f"{timestamp}_desde_punto_{ORIGIN_POINT_ID}"
            SESSION_OUTPUT_DIR = os.path.join(BASE_PROCESSING_FOLDER, session_folder_name)
            log.info(f"MODO NUEVO: Creando nueva sesión: {session_folder_name}")
        os.makedirs(SESSION_OUTPUT_DIR, exist_ok=True)
        with rasterio.open(COST_RASTER_PATH) as src:
            log.info("Validando datos y extrayendo metadatos del raster...")
            if MASK_SHAPEFILE_PATH:
                mask_polygon_geom = load_polygon_geometry(MASK_SHAPEFILE_PATH)
                if mask_polygon_geom:
                    log.info("Verificando puntos contra la máscara...")
                    invalid_points_ids = [str(pid) for pid, coords in all_points.items() if not mask_polygon_geom.contains(Point(coords))]
                    if invalid_points_ids:
                        log.critical(f"ANÁLISIS CANCELADO! Puntos fuera de máscara: {', '.join(invalid_points_ids)}"); exit()
                    log.info("VERIFICACIÓN SUPERADA.")
            origin_coords = all_points.get(ORIGIN_POINT_ID)
            if origin_coords is None:
                log.critical(f"¡ANÁLISIS CANCELADO! El punto de origen con ID '{ORIGIN_POINT_ID}' no se encontró en '{ID_FIELD_NAME}'.")
                log.critical(f"IDs disponibles: {sorted(list(all_points.keys()))}"); exit()
            start_pixel_hr = world_to_pixel(src.transform, origin_coords[0], origin_coords[1])
            log.info(f"Origen Fijo: Punto ID {ORIGIN_POINT_ID} -> Píxel {start_pixel_hr}")
            shared_data = {
                'origin_id': ORIGIN_POINT_ID, 'start_pixel_hr': start_pixel_hr,
                'cost_raster_path': COST_RASTER_PATH, 'mask_shapefile_path': MASK_SHAPEFILE_PATH,
                'transform': src.transform, 'crs': src.crs, 'nodata': src.nodata, 'res': src.res, 'shape': src.shape,
                'session_output_dir': SESSION_OUTPUT_DIR, 'downsampling_factors': DOWNSAMPLING_FACTORS,
                'corridor_buffer_pixels': CORRIDOR_BUFFER_PIXELS, 'heuristic_weight': HEURISTIC_WEIGHT
            }
        tasks_to_run = []
        for dest_id, dest_coords in all_points.items():
            if dest_id == ORIGIN_POINT_ID: continue
            final_output_path = os.path.join(SESSION_OUTPUT_DIR, f"ruta_final_desde_{ORIGIN_POINT_ID}_a_{dest_id}.shp")
            if os.path.exists(final_output_path):
                log.info(f"Ruta para {ORIGIN_POINT_ID}->{dest_id} ya existe. Omitiendo.")
                continue
            tasks_to_run.append((dest_id, dest_coords))
        log.info(f"Se procesarán {len(tasks_to_run)} rutas pendientes en paralelo.")
        if tasks_to_run:
            full_tasks = [(dest_id, dest_coords, shared_data) for dest_id, dest_coords in tasks_to_run]
            stop_event = threading.Event(); monitor_thread = threading.Thread(target=resource_monitor, args=(stop_event, monitoring_records))
            start_time = time.time()
            monitor_thread.start()
            results = Parallel(n_jobs=N_JOBS)(
                delayed(calculate_route_worker)(task) for task in tqdm(full_tasks, desc="Calculando rutas paralelas")
            )
            stop_event.set(); monitor_thread.join()
            total_duration = time.time() - start_time
            log.info(f"Proceso paralelo completado en {total_duration:.2f} segundos.")
            exitos = [r for r in results if r[0] == "Éxito"]
            log.info(f"RESUMEN: {len(exitos)} de {len(tasks_to_run)} rutas calculadas con éxito.")
        else:
            log.info("No hay nuevas rutas que calcular en esta sesión.")
    except Exception as e:
        logging.critical(f"Error fatal en el proceso principal: {e}", exc_info=True)
    # Reporte exportación de rutas
    # Loop para verificar que el usuario haya puesto la opción correcta, y en el caso de que no, dar la opción nuevamente para escribirla
    while(True):
        if opcionRutas in ['Y', 'y']:
            final_route_files = glob.glob(os.path.join(OUTPUT_DIR, 'ruta_final_*.shp'))
            if not final_route_files:
                print("No se encontraron archivos de rutas finales para unir.")
                break
            else:
                final_routes_gdf = pd.concat([gpd.read_file(f) for f in final_route_files], ignore_index=True)
                # Exportar archivo
                # Problema -> no exporta a la carpeta específica de sesión, sino que a la carpeta output misma
                final_routes_gdf.to_file(OUTPUT_DIR+'.shp', driver='ESRI Shapefile')
                print(f"Rutas unidas y exportadas a {OUTPUT_DIR}.shp")
                break
        if opcionRutas in ['n', 'N']:
            print("No se exportará el archivo unido.")
            break
        else:
            opcionRutas = input("Argumento no identificado, usar solo Y/N")
    finally:
        if monitoring_records:
            avg_cpu = np.mean([r['cpu'] for r in monitoring_records])
            peak_mem = np.max([r['mem_mb'] for r in monitoring_records])
            log.info("\n--- Reporte de Rendimiento ---")
            log.info(f"Uso promedio de CPU: {avg_cpu:.2f}%")
            log.info(f"Pico de uso de Memoria: {peak_mem:.2f} MB")
        log.info("================== EJECUCIÓN FINALIZADA ==================\n\n")


In [ ]:
# ==============================================================================
# --- CELDA 6: VISUALIZACIÓN DE LA RED COMPLETA (TODOS CON TODOS) ---
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import geopandas as gpd
import pandas as pd
import rasterio
import rasterio.plot
from rasterio.windows import from_bounds
import glob

print("Iniciando la visualización de la red completa de rutas...")

try:
    # --- 1. Encontrar y cargar TODAS las rutas generadas ---
    # MODIFICACIÓN: El patrón glob ahora busca cualquier ruta final, sin especificar origen.
    final_route_files = glob.glob(os.path.join(OUTPUT_DIR, 'ruta_final_*.shp'))

    if not final_route_files:
        print("No se encontraron archivos de rutas finales para visualizar.")
    else:
        # Cargar todas las rutas en un único GeoDataFrame
        final_routes_gdf = pd.concat([gpd.read_file(f) for f in final_route_files], ignore_index=True)
        
        # --- 2. Determinar la extensión total con margen ---
        minx, miny, maxx, maxy = final_routes_gdf.total_bounds
        margin = 100
        buffered_bounds = (minx - margin, miny - margin, maxx + margin, maxy + margin)
        
        # --- 3. Cargar la porción del raster ---
        with rasterio.open(COST_RASTER_PATH) as src:
            window = from_bounds(*buffered_bounds, transform=src.transform)
            raster_data = src.read(1, window=window)
            window_transform = src.window_transform(window)

        # --- 4. Cargar todos los puntos ---
        points_gdf = gpd.read_file(ALL_POINTS_SHAPEFILE)

        # --- 5. Crear el mapa ---
        fig, ax = plt.subplots(figsize=(18, 18))
        rasterio.plot.show(raster_data, transform=window_transform, ax=ax, cmap='viridis', norm=LogNorm(), alpha=0.8)
        
        # MODIFICACIÓN: Dibujar todas las rutas con alta transparencia y líneas finas
        final_routes_gdf.plot(
            ax=ax,
            edgecolor='red',
            linewidth=0.8,  # Líneas más finas
            label='Rutas Calculadas',
            alpha=0.5,      # Alta transparencia para ver superposiciones
            zorder=3
        )
        
        # MODIFICACIÓN: Dibujar todos los puntos con el mismo estilo
        points_gdf.plot(
            ax=ax,
            color='yellow',
            markersize=50,
            ec='black',
            label='Nodos de la Red',
            zorder=5
        )

        # Configuración final del mapa
        ax.set_title('Red Completa de Rutas de Menor Costo', fontsize=20)
        ax.set_xlabel("Coordenada X"); ax.set_ylabel("Coordenada Y")
        ax.legend(); plt.grid(True, linestyle='--', alpha=0.5); plt.tight_layout(); plt.show()
        # Configuración final del mapa
        ax.set_title('Red Completa de Rutas de Menor Costo', fontsize=20)
        ax.set_xlabel("Coordenada X"); ax.set_ylabel("Coordenada Y")
        ax.legend(); plt.grid(True, linestyle='--', alpha=0.5); plt.tight_layout(); plt.show()

        # Preguntar al usuario si quiere exportar el mapa
        while(True):
            opcion = str(input("Exportar el mapa a la carpeta /output? (Y/N)"))
            if opcion in ['y', 'Y']:
                print("Exportando mapa...")
                fig.savefig(OUTPUT_DIR)
                print(f"{OUTPUT_DIR}.jpg exportado exitosamente a /output")
                break
            if opcion in ['n', 'N']:
                print("No se exportara el mapa")
                break
            else:
                print("Argumento no identificado, usar solo Y/N")

except Exception as e:
    print(f"Ocurrió un error durante la visualización: {e}")
    import traceback
    traceback.print_exc()